In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the CSV file
df = pd.read_csv('docs/owid-covid-data.csv')

# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter the data for the year 2021
df_2021 = df[df['date'].dt.year == 2021]

# Function to fill NaN values with the last available non-NaN value per country
def fill_last_available(df, col):
    return df.groupby('location')[col].fillna(method='ffill')

# Fill NaN values for the relevant columns
df_2021['excess_mortality_cumulative_per_million'] = fill_last_available(df_2021, 'excess_mortality_cumulative_per_million')
df_2021['people_fully_vaccinated_per_hundred'] = fill_last_available(df_2021, 'people_fully_vaccinated_per_hundred')

# Extract the last available data for each country
df_last_2021 = df_2021.groupby('location').last().reset_index()

# Create scatter plot for excess deaths vs people fully vaccinated
fig1 = px.scatter(df_last_2021, x='people_fully_vaccinated_per_hundred', y='excess_mortality_cumulative_per_million', 
                 trendline='ols', trendline_color_override='darkblue',
                 title='Excess mortality per million inhabitants vs. People fully vaccinated per hundred inhabitants, per country for 2021',
                 labels={'people_fully_vaccinated_per_hundred': 'People fully vaccinated per hundred inhabitants',
                         'excess_mortality_cumulative_per_million': 'Excess mortality cumulative per million inhabitants'},
                 hover_name='location', opacity=0.7)

# Grab the GDP per capita data for the last day of 2021
df_gdp = df[df['date'] == '2021-12-31']

# Create scatter plot for GDP vs people fully vaccinated
fig2 = px.scatter(df_gdp, x='people_fully_vaccinated_per_hundred', y=df.columns[52], 
                 trendline='ols', trendline_color_override='darkblue',
                 title='GDP per capita vs. People fully vaccinated per hundred inhabitants, per country for 2021',
                 labels={'people_fully_vaccinated_per_hundred': 'People fully vaccinated per hundred inhabitants',
                         df.columns[52]: 'GDP per capita'},
                 hover_name='location', opacity=0.7)

# Create a figure with subplots
fig = go.Figure()

# Add traces for the first plot
for trace in fig1.data:
    fig.add_trace(trace)

# Add traces for the second plot
for trace in fig2.data:
    fig.add_trace(trace)

# Update layout for buttons to toggle between plots
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=[{"visible": [True] * len(fig1.data) + [False] * len(fig2.data)},
                          {"title": "Excess mortality per million inhabitants vs. People fully vaccinated per hundred inhabitants, per country for 2021"}],
                    label="Excess Deaths vs Vaccinations",
                    method="update"
                ),
                dict(
                    args=[{"visible": [False] * len(fig1.data) + [True] * len(fig2.data)},
                          {"title": "GDP per capita vs. People fully vaccinated per hundred inhabitants, per country for 2021"}],
                    label="GDP vs Vaccinations",
                    method="update"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.57,
            xanchor="center",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# Initially set the second plot to be not visible
for i in range(len(fig2.data)):
    fig.data[len(fig1.data) + i].visible = False

fig.show()


/tmp/ipykernel_84281/1991951373.py:16: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  return df.groupby('location')[col].fillna(method='ffill')
/tmp/ipykernel_84281/1991951373.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.groupby('location')[col].fillna(method='ffill')
/tmp/ipykernel_84281/1991951373.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['excess_mortality_cumulative_per_million'] = fill_last_available(df_2021, 'excess_mortality_cumulative

ModuleNotFoundError: No module named 'statsmodels'